In [1]:
import numpy as np
import pandas as pd
import sympy as sp

from utilities import *

In [2]:
SLOPE_ANGLE = 3 * sp.pi / 360
DETECTOR_ANGLE = 2 * sp.pi / 3
DIRECTION_ANGLE = sp.pi/2
CENTER_DEPTH = 70

PRECISION = 20

In [3]:
# 创建一个 dataframe 用于存储数据
# 四列分别为 dis_from_center, water_depth, scan_width, ocerlap_ratio
df = pd.DataFrame(columns=['dis_from_center', 'water_depth', 'scan_width', 'ocerlap_ratio'])
# dis_from_center 值从 -800 到 800，步长为 200
df['dis_from_center'] = np.arange(-800, 1000, 200)

df

,dis_from_center,water_depth,scan_width,ocerlap_ratio
0,-800,NaN,NaN,NaN
1,-600,NaN,NaN,NaN
2,-400,NaN,NaN,NaN
3,-200,NaN,NaN,NaN
4,0,NaN,NaN,NaN
5,200,NaN,NaN,NaN
6,400,NaN,NaN,NaN
7,600,NaN,NaN,NaN
8,800,NaN,NaN,NaN


In [4]:
field = Field(1000, 1000, CENTER_DEPTH, SLOPE_ANGLE, PRECISION)
detectors = []

for i in range(len(df)):
    # 创建一个 Detector 对象
    detector = Detector(field,
                        -df.loc[i, 'dis_from_center'],
                        0,
                        DIRECTION_ANGLE,
                        DETECTOR_ANGLE,
                        PRECISION)
    df.loc[i, 'water_depth'] = detector.loc_depth

    detector.get_scan_segment()
    detectors.append(detector)
    # 计算扫描宽度
    df.loc[i, 'scan_width'] = detector.scan_segment_length_proj_xy

    if i > 0:
        # 计算重叠率
        slope_length_between_depth_point = (df.loc[i, 'dis_from_center'] - df.loc[i-1, 'dis_from_center']) / sp.cos(SLOPE_ANGLE)
        slope_length_half_pre = sp.Segment(detectors[i-1].depth_point, detectors[i-1].detect_point_1).length.evalf(PRECISION)
        slope_length_half_cur = sp.Segment(detectors[i].depth_point, detectors[i].detect_point_2).length.evalf(PRECISION)
        df.loc[i, 'ocerlap_ratio'] = ((slope_length_half_pre + slope_length_half_cur - slope_length_between_depth_point) / slope_length_between_depth_point).evalf(PRECISION)

df

,dis_from_center,water_depth,scan_width,ocerlap_ratio
0,-800,90.948737255349543112,315.70510695333724903,NaN
1,-600,85.711552941512157334,297.52557105398511351,0.53101534713891183217
2,-400,80.474368627674771556,279.34603515463297798,0.44011766764215115455
3,-200,75.237184313837385778,261.16649925528084246,0.34921998814539047694
4,0,70,242.98696335592870694,0.25832230864862979932
5,200,64.762815686162614222,224.80742745657657141,0.16742462915186912168
6,400,59.525631372325228444,206.62789155722443587,0.076526949655108444043
7,600,54.288447058487842666,188.44835565787230035,-0.014370729841652233615
8,800,49.051262744650456888,170.26881975852016483,-0.10526840933841291123


In [5]:
df.to_csv('1.csv', index=False)